In [ ]:
import csv
import sys

ASDIR = "../usb/ovk/data/eriktks/AS/text"
ESDIR = "../usb/ovk/data/eriktks/ES/text"
REVERSEDFILENAME = "reversed.txt"
MAILFILENAME = "mails.csv"
SEPARATOR = ","
CLIENT = "client-id"
COUNSELOR = "counselor"
DATE = "date"
SENDER = "sender"

In [ ]:
def readFirstMails(inDir):
    mails = {}
    inFileName = inDir+"/"+MAILFILENAME
    try: inFile = open(inFileName,"r")
    except Exception as e: sys.exit("cannot read file "+inFileName+" "+str(e))
    csvReader = csv.DictReader(inFile,delimiter=SEPARATOR)
    for row in csvReader:
        try:
            client = row[CLIENT]
            date = row[DATE]
            sender = row[SENDER]
            if sender == "CLIENT" and \
               (not client in mails or 
                (mails[client][DATE] > date and date != "")):
                mails[client] = dict(row)
        except Exception as e: sys.exit("error: "+str(e))      
    inFile.close()
    return(mails)
asMails = readFirstMails(ASDIR)
esMails = readFirstMails(ESDIR)

In [ ]:
tokens = {}
for client in asMails.keys():
    mailTokens = {}
    for token in asMails[client]["text"].split(): mailTokens[token] = True
    for token in mailTokens.keys():
        if token in tokens: tokens[token] += 1
        else: tokens[token] = 1
features = list(tokens.keys())
featureIds = {}
for f in range(0,len(features)): featureIds[features[f]] = f

In [ ]:
OUTFILENAME="data2tsne.txt"
try: outFile = open(OUTFILENAME,"w")
except Exception as e: sys.exit("error: cannot write file "+OUTFILENAME+": "+str(e))
for client in asMails.keys():
    mailTokens = {}
    for token in asMails[client]["text"].split(): mailTokens[token] = True
    for i in range(0,len(features)):
        if features[i] in mailTokens: print(1.0,end=" ",file=outFile)
        else: print(0.0,end=" ",file=outFile)
    print("",file=outFile)
outFile.close()